# Bibliotecas

In [5]:
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import random

# Diretório

In [6]:
# Diretórios principais
base_dir = "D:\\T.I\\BootcampAvanti_ML\\DeteccaoMelanoma_ML"
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Subdiretórios de classe
classes = ['Benign', 'Malignant']

# Diretórios para salvar as máscaras
mask_dir_train = os.path.join(base_dir, 'masks_R-CNN_train')
mask_dir_test = os.path.join(base_dir, 'masks_R-CNN_test')


# Modelo Mask R-CNN

In [7]:
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2 import model_zoo

# Configuração do Mask R-CNN
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Threshold para as predições
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'  # Força o uso da CPU
predictor = DefaultPredictor(cfg)


# Funções para processamento

#### Função simples

In [5]:
# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para gerar máscaras usando Mask R-CNN
def generate_masks(image):
    outputs = predictor(image)
    masks = outputs["instances"].pred_masks.cpu().numpy()
    return masks

# Função para salvar as máscaras
def save_masks(masks, output_dir, img_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, mask in enumerate(masks):
        mask_image = Image.fromarray((mask * 255).astype(np.uint8))
        mask_image.save(os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}_mask_{i}.png"))

# Função para processar imagens de um diretório e salvar máscaras
def process_images(input_dir, output_dir, sample_size=None):
    for class_name in classes:
        class_dir = os.path.join(input_dir, class_name)
        dest_mask_dir = os.path.join(output_dir, class_name)
        
        if not os.path.exists(dest_mask_dir):
            os.makedirs(dest_mask_dir)

        # Iterar sobre todas as imagens ou pegar uma amostra
        img_names = os.listdir(class_dir)
        if sample_size:
            img_names = img_names[:sample_size]
        
        for img_name in img_names:
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):
                try:
                    # Carregar e preparar a imagem
                    img = load_image(img_path)
                    
                    # Gerar máscaras
                    masks = generate_masks(img)
                    
                    # Salvar as máscaras
                    save_masks(masks, dest_mask_dir, img_name)
                    
                    print(f"Máscaras salvas para {img_name} em {dest_mask_dir}")
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_name}: {e}")





In [6]:
# Processar imagens do conjunto de treinamento (amostra de 3 imagens)
process_images(train_dir, mask_dir_train, sample_size=200)

# Processar imagens do conjunto de teste (amostra de 3 imagens)
process_images(test_dir, mask_dir_test, sample_size=100)



C:\Users\User\AppData\Roaming\Python\Python311\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Máscaras salvas para 1.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 10.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 100.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1000.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1001.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1002.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1003.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1004.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1005.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Máscaras salvas para 1006.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\m

# Funções para o processamento com traqquindo do tempo

In [8]:
import os
import time
import numpy as np
from PIL import Image

# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para gerar máscaras usando Mask R-CNN
def generate_masks(image):
    outputs = predictor(image)
    masks = outputs["instances"].pred_masks.cpu().numpy()
    return masks

# Função para salvar as máscaras
def save_masks(masks, output_dir, img_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, mask in enumerate(masks):
        mask_image = Image.fromarray((mask * 255).astype(np.uint8))
        mask_image.save(os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}_mask_{i}.png"))

# Função para processar imagens de um diretório e salvar máscaras
def process_images(input_dir, output_dir, sample_size=None):
    total_start_time = time.time()  # Tempo total de início
    total_processed_images = 0  # Contador de imagens processadas
    
    for class_name in classes:
        class_dir = os.path.join(input_dir, class_name)
        dest_mask_dir = os.path.join(output_dir, class_name)
        
        if not os.path.exists(dest_mask_dir):
            os.makedirs(dest_mask_dir)

        # Iterar sobre todas as imagens ou pegar uma amostra
        img_names = os.listdir(class_dir)
        if sample_size:
            img_names = img_names[:sample_size]
        
        for img_name in img_names:
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):
                img_start_time = time.time()  # Tempo de início para a imagem atual
                
                try:
                    # Carregar e preparar a imagem
                    img = load_image(img_path)
                    
                    # Gerar máscaras
                    masks = generate_masks(img)
                    
                    # Salvar as máscaras
                    save_masks(masks, dest_mask_dir, img_name)
                    
                    img_end_time = time.time()  # Tempo de término para a imagem atual
                    img_processing_time = img_end_time - img_start_time
                    
                    total_processed_images += 1  # Incrementar o contador
                    
                    print(f"Máscaras salvas para {img_name} em {dest_mask_dir}")
                    print(f"Tempo de processamento da imagem {img_name}: {img_processing_time:.2f} segundos")
                
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_name}: {e}")

    total_end_time = time.time()  # Tempo total de término
    total_processing_time = total_end_time - total_start_time
    
    print(f"Tempo total de processamento: {total_processing_time:.2f} segundos")
    print(f"Total de imagens processadas: {total_processed_images}")

# Exemplo de uso
# process_images(input_dir="path_to_input_dir", output_dir="path_to_output_dir", sample_size=None)


In [9]:
# Processar imagens do conjunto de treinamento (amostra de 3 imagens)
process_images(train_dir, mask_dir_train, sample_size=None)

# Processar imagens do conjunto de teste (amostra de 3 imagens)
process_images(test_dir, mask_dir_test, sample_size=None)

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Máscaras salvas para 1.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 1.jpg: 3.71 segundos
Máscaras salvas para 10.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 10.jpg: 3.50 segundos
Máscaras salvas para 100.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 100.jpg: 3.58 segundos
Máscaras salvas para 1000.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 1000.jpg: 3.60 segundos
Máscaras salvas para 1001.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 1001.jpg: 3.46 segundos
Máscaras salvas para 1002.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_R-CNN_train\Benign
Tempo de processamento da imagem 1002.jpg: 3.60 segundos
Máscaras salvas para 1003.jpg em D:\T.I\BootcampAvan

# Função para visualizar imagem e mascara

In [6]:

# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para visualizar a imagem e sua máscara
def visualize_images_and_masks(mask_dir, base_dir, sample_size=5):
    # Listar todos os arquivos de máscara
    mask_files = []
    for class_name in classes:
        class_mask_dir = os.path.join(mask_dir, class_name)
        if os.path.exists(class_mask_dir):
            mask_files.extend([os.path.join(class_mask_dir, f) for f in os.listdir(class_mask_dir) if f.endswith('.png')])

    # Selecionar uma quantidade aleatória de máscaras
    sampled_masks = random.sample(mask_files, min(sample_size, len(mask_files)))
    
    # Criar a visualização
    for mask_file in sampled_masks:
        # Extrair o nome da imagem original do arquivo de máscara
        img_name = os.path.basename(mask_file).split('_mask_')[0]
        
        # Procurar a imagem original no diretório base correspondente
        for class_name in classes:
            img_path = os.path.join(base_dir, class_name, f"{img_name}.jpg")
            if os.path.isfile(img_path):
                # Carregar e preparar a imagem original
                img = load_image(img_path)
                
                # Carregar a máscara
                mask = Image.open(mask_file).convert("L")
                mask = np.array(mask)

                # Plotar imagem original e máscara
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.imshow(img)
                plt.title(f"Imagem Original: {img_name}")
                plt.axis('off')

                plt.subplot(1, 2, 2)
                plt.imshow(mask, cmap='gray')
                plt.title(f"Máscara: {img_name}")
                plt.axis('off')

                plt.show()
                break

# Usar a função para visualizar imagens e máscaras
visualize_images_and_masks(mask_dir_train, train_dir, sample_size=20)
visualize_images_and_masks(mask_dir_test, test_dir, sample_size=10)


: 

In [10]:
# Plot Xique


import os
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para visualizar a imagem e sua máscara
def visualize_images_and_masks(mask_dir, base_dir, sample_size=5):
    # Listar todos os arquivos de máscara
    mask_files = []
    for class_name in classes:
        class_mask_dir = os.path.join(mask_dir, class_name)
        if os.path.exists(class_mask_dir):
            mask_files.extend([os.path.join(class_mask_dir, f) for f in os.listdir(class_mask_dir) if f.endswith('.png')])

    # Selecionar uma quantidade aleatória de máscaras
    sampled_masks = random.sample(mask_files, min(sample_size, len(mask_files)))
    
    # Criar a visualização
    num_samples = len(sampled_masks)
    num_rows = (num_samples * 2 + 1) // 2
    plt.figure(figsize=(15, num_rows * 6))  # Ajuste o tamanho da figura conforme necessário

    for i, mask_file in enumerate(sampled_masks):
        # Extrair o nome da imagem original do arquivo de máscara
        img_name = os.path.basename(mask_file).split('_mask_')[0]
        
        # Procurar a imagem original no diretório base correspondente
        for class_name in classes:
            img_path = os.path.join(base_dir, class_name, f"{img_name}.jpg")
            if os.path.isfile(img_path):
                # Carregar e preparar a imagem original
                img = load_image(img_path)
                
                # Carregar a máscara
                mask = Image.open(mask_file).convert("L")
                mask = np.array(mask)

                # Plotar imagem original
                plt.subplot(num_rows, 2, 2 * i + 1)
                plt.imshow(img)
                plt.title(f"Imagem Original: {img_name}")
                plt.axis('off')

                # Plotar máscara
                plt.subplot(num_rows, 2, 2 * i + 2)
                plt.imshow(mask, cmap='gray')
                plt.title(f"Máscara: {img_name}")
                plt.axis('off')

                break

    plt.tight_layout()
    plt.show()

# Usar a função para visualizar imagens e máscaras
visualize_images_and_masks(mask_dir_train, train_dir, sample_size=5)
visualize_images_and_masks(mask_dir_test, test_dir, sample_size=5)


: 